# Algebraic Geometry in OSCAR: part I

[OSCAR](https://www.computeralgebra.de) is a computer algebra system whose development is funded by the Deutsche Forschungsgemeinschaft (DFG) as part of the SFB-TRR 195. It is open source and can be found on [GitHub](www.github.com/oscar-system/Oscar.jl). The core functionality is for all kinds of *symbolic computations* and comprises number theory, group and representation theory, polyhedral geometry, commutative algebra, ...

Today, we will have a look at how to use OSCAR to do Algebraic Geometry and, in particular, Intersection Theory. We will mostly follow **[Ful98]** and the [stacks project](https://stacks.math.columbia.edu/).

### Installation and starting up OSCAR 

Installation instructions for OSCAR can be found on its [website](https://www.oscar-system.org/install/). 
For the purpose of this course, we shall need a very particular branch of OSCAR which can be installed using the following commands.

In [ ]:
using Pkg
]add https://github.com/HechtiDerLachs/Oscar.jl#Durham_2024

To use OSCAR, start up `julia` and type

In [ ]:
#using Revise, Oscar # Revise is another very useful, independent package 
                     # which allows you to change code without restarting.
using Oscar

You may need to do some updates, eventually:

In [ ]:
]up

### First steps for Algebraic Geometry in OSCAR

Let us define an elliptic curve $E$ over the function field $\mathbb k(t)$ for $\mathbb k = \mathbb F_{113}$.

In [ ]:
kk = GF(113) # the finite field (Galois field) with 113 elements

In [ ]:
kkt_poly, t_poly = kk[:t] # the polynomial ring in one variable `t` over `kk`

In [ ]:
kkt_field = fraction_field(kkt_poly) # the fraction field of that polynomial ring

In [ ]:
t_field = kkt_field(t_poly) # the monomial `t`, but as an element of the fraction field

In [ ]:
P, (x, y) = kkt_field[:x, :y]; # the bivariate polynomial ring over `kkt_field`
P

In [ ]:
f = y^2 - x^3 - (t_poly^8 - 1)*x # a Weierstrass equation for an elliptic curve

In [ ]:
E = elliptic_curve(f, x, y) # the associated elliptic curve as an instance of `EllipticCurve`

### Affine schemes in OSCAR

For an elliptic curve as above with *integral* coefficients, we can build the *Weierstrass model* of its associated *elliptic surface*.

In [ ]:
P, (x, y, t) = kk[:x, :y, :t] # a multivariate polynomial ring in three variables
f = y^2 - x^3 - (t^8 - 1)*x # the defining equation for the Weierstrass chart
I = ideal(P, f) # a principal ideal in `P`

To any finitely generated `kk`-algebra we can associate its [spectrum](https://stacks.math.columbia.edu/tag/01HR).

In [ ]:
IA3 = spec(P) # the affine scheme associated to `P`
W, inc_W = sub(IA3, I) # the closed subscheme defined by `I` together with its `ClosedEmbedding`
W

In [ ]:
W_sing, inc_W_sing = singular_locus(W) # the singular locus of `W` with its `ClosedEmbedding`
W_sing

The second return value is a [closed embedding](https://stacks.math.columbia.edu/tag/01HJ) which has its own datatype and its own methods in OSCAR.

In [ ]:
typeof(inc_W_sing)

In [ ]:
methodswith(ClosedEmbedding)

In [ ]:
I_sing = image_ideal(inc_W_sing) # the ideal defining `W_sing` as a closed subscheme of `W`

We can get our hands on the distict points of the singular locus via a primary decomposition.

In [ ]:
I_sing_dec = primary_decomposition(I_sing) # eight distinct points at `(x, y, t) = (0, 0, \zeta_8^k)`

In OSCAR we can work with polynomial rings and their quotients, but also with *localizations*.

In [ ]:
U = hypersurface_complement(W, t^8-1)

In [ ]:
Sigma, _ = singular_locus(U)
is_empty(Sigma)

### Covered schemes in OSCAR

The above were examples of *affine schemes* and their morphisms. These are but wrappers of objects in commutative algebra. But in order to construct the actual elliptic surface associated to $E$, we need to resolve the singularities of the Weierstrass model $W$. 

This will lead to *covered schemes* which are glued from affine schemes as for example described [here](https://stacks.math.columbia.edu/tag/01JA).

In [ ]:
P1 = I_sing_dec[1][1] # an ideal in `OO(W)`
pr1 = blow_up(W, P1) # the projection morphism `W1 -> W`

We have just [blown up](https://stacks.math.columbia.edu/tag/052P) the affine scheme `W` along the ideal `P1`.

In [ ]:
W1 = domain(pr1); # a `CoveredScheme` over `kk`
cov = default_covering(W1) # the initial `Covering` used to define the scheme

In [ ]:
U = patches(cov)[1] # an affine scheme; the first chart of `W1`

In [ ]:
V = patches(cov)[2]

In [ ]:
glue = cov[U, V] # either two `patches` in a `Covering` might be glued along open subsets

In [ ]:
U_simp = simplify(U) # one can always try to eliminate variables using `simplify`

In [ ]:
E1 = exceptional_divisor(pr1) # the exceptional divisor of the blowup
typeof(E1)

In [ ]:
E1(U_simp) # the defining equations for the exceptional divisor in `U_simp`

In [ ]:
small_generating_set(ideal_sheaf(E1)(U)) # a glimpse of the ideal in the whole affine ambient space

In [ ]:
is_prime(ideal_sheaf(E1)) # in this case the exceptional divisor is prime

We can perform (parts of) the usual operations for the blowup morphism `pr1`, such as e.g. pullback:

In [ ]:
W_cov = codomain(pr1) # a `CoveredScheme` with only one chart `W`
II = ideal_sheaf(W_cov, W, ideal(OO(W), t+1)) # the `AbsIdealSheaf` on `W_cov` for the fiber over `t = -1

In [ ]:
pb1_II = pullback(pr1, II) # the ideal sheaf for the fiber over `t = -1` in the blowup `W1`
pb1_II(U_simp)

In [ ]:
is_prime(pb1_II)

In [ ]:
F1 = weil_divisor(pb1_II) # the fiber as a `AbsWeilDivisor`
F1 = irreducible_decomposition(F1) # ...decomposed into irreducible components

In [ ]:
Q1, Q2 = components(F1);

Let us compare the components of the fiber with the exceptional divisor and the strict transform of the fiber.

In [ ]:
str1_II = strict_transform(pr1, II);
is_prime(str1_II)

In [ ]:
ideal_sheaf(E1) == Q1

In [ ]:
ideal_sheaf(E1) == Q2

In [ ]:
str1_II == Q1

In [ ]:
str1_II == Q2

For the fun of it, let us have a look at the `weil_divisor` associated to the square of the fiber ideal

In [ ]:
JJ = pb1_II^2
@time irreducible_decomposition(weil_divisor(JJ))

How do the components of the fiber over `t = -1` intersect?

In [ ]:
intersect(weil_divisor(Q1), weil_divisor(Q2)) # A custom method for Weil divisors on surfaces!

How do we actually get this number?

In [ ]:
JJ = Q1 + Q2; # the sum of two ideal sheaves; will be 0-dimensional in this case
Z = AlgebraicCycle(W1, ZZ, IdDict([JJ=>one(ZZ)])) # the associated `AbsAlgebraicCycle` with ZZ-coefficients

In [ ]:
irreducible_decomposition(Z) # decomposition into prime components with multiplicities

In [ ]:
integral(ans) # pushforward to the point `spec(kk)`

# References

 * **[Ful98]**: Fulton, W., Intersection Theory, Springer, 1998